In [ ]:
# Import the dependencies
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from google.colab import drive


Using TensorFlow backend.


We load the song dataset and filter by the name of the song and lyrics.


In [ ]:
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/DeepLearning_2020/test/songdata.csv'

dataset = pd.read_csv(data_path)
print("Our dataset contains "+str(dataset.shape[0])+" song lyrics.")
#print(dataset.columns)
print("of "+str(len(dataset['artist'].unique()))+ " different artists")
datas = dataset.filter(['song','text'],axis=1)
#for col in datas.columns:
#  print(col)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Our dataset contains 57650 song lyrics.
of 643 different artists


In [ ]:
datas.head()

,song,text
0,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd..."
1,"Andante, Andante","Take it easy with me, please \nTouch me gentl..."
2,As Good As New,I'll never know why I had to go \nWhy I had t...
3,Bang,Making somebody happy is a question of give an...
4,Bang-A-Boomerang,Making somebody happy is a question of give an...


To have all the lyrics as a training set, we will write in a txt file all the lyrics of all songs. This will be useful for training data in the LSTM model.

In [ ]:
lyrics_data = pd.DataFrame({'songID': dataset.index, 'songName':dataset['song'], 'lyrics':dataset['text'] })
raw_text = ','.join(lyrics_data['lyrics'])
raw_text = raw_text.lower()
lyrics_data.head()

,songID,songName,lyrics
0,0,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd..."
1,1,"Andante, Andante","Take it easy with me, please \nTouch me gentl..."
2,2,As Good As New,I'll never know why I had to go \nWhy I had t...
3,3,Bang,Making somebody happy is a question of give an...
4,4,Bang-A-Boomerang,Making somebody happy is a question of give an...


As we will perform a char-level RNN and LSTM, we are going to store all unique characters and get the size of the vocabulary we are using. We will also map each character to an index.

In [ ]:
chars = sorted(list(set(raw_text)))
vocabulary_size = len(chars)

char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

RNN model

In [ ]:
import tensorflow as tf
import random
tf.compat.v1.disable_eager_execution()
data = raw_text

Given an index of a character, returns the one-hot encoded vectors.

In [ ]:
def one_hot_encoder(index):
    return np.eye(vocabulary_size)[index]

Now we define the RNN network parameters.

In [ ]:
#units in the hidden layer
hidden_size = 120
#length output
seq_length = 200  
#define learning rate for gradient descent is as follows:
learning_rate = 0.0000000001
#set the seed value:
seed_value = 42
tf.compat.v1.set_random_seed(seed_value)
random.seed(seed_value)

inputs =tf.compat.v1.placeholder(shape=[None, vocabulary_size],dtype=tf.float32, name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocabulary_size], dtype=tf.float32, name="targets")
init_state = tf.compat.v1.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")
initializer = tf.compat.v1.random_normal_initializer(stddev=0.1)

Forward propagation:


In [ ]:
with tf.compat.v1.variable_scope("RNN") as scope:
  h_t = init_state
  y_hat = []

  for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
      if t > 0:
          scope.reuse_variables()  

      #input to hidden layer weights
      U = tf.compat.v1.get_variable("U", [vocabulary_size, hidden_size], initializer=initializer)
      #hidden to hidden layer weights
      W = tf.compat.v1.get_variable("W", [hidden_size, hidden_size], initializer=initializer)
      #output to hidden layer weights
      V = tf.compat.v1.get_variable("V", [hidden_size, vocabulary_size], initializer=initializer)
      #bias for hidden layer
      bh = tf.compat.v1.get_variable("bh", [hidden_size], initializer=initializer)
      #bias for output layer
      by = tf.compat.v1.get_variable("by", [vocabulary_size], initializer=initializer)
      h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
      y_hat_t = tf.matmul(h_t, V) + by
      y_hat.append(y_hat_t)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Now we apply the activation function (softmax).

In [ ]:
output_softmax = tf.nn.softmax(y_hat[-1])  
outputs = tf.concat(y_hat, axis=0)
#compute the loss and store the final hidden state in hprev --> we will use it for the predictions
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))
hprev = h_t

We need to compute the gradients of the loss using the Adam optimizer.

In [ ]:
minimizer = tf.compat.v1.train.AdamOptimizer()
gradients = minimizer.compute_gradients(loss)
#there are gradients that exceed the threshold --> clipped!
threshold = tf.constant(5.0, name="grad_clipping")
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

updated_gradients = minimizer.apply_gradients(clipped_gradients)

We start running the model with the tensorflow session.

In [ ]:
sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)
#points the character in our dataset
pointer = 0
iteration = 0
maxiters = 20000
while iteration < maxiters:
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence --> slice the data
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocabulary_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %1000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 5000 iterations

  tree  
and i fatte it bean  
out e'vemy,cruss i'll arous diys a's can's jukt sifan to su-d!if urees  
  
how on't or thitt'rca 

yon  
it's bisthour laxt a aline sonellin' s no't a beacs  
widbe leanges ars  
 to gera mying tt the peawoon, it ig be cat be turs  
you meve i ffill taids  
 in laymay ma knought se laken tine "sen't  
nut my fomrty ball code soby a lank could pike orking!s  
our ille tarul wer, lare ba chay don'g greroag saike!..  
the handel caice hancat: a wo beer your lopcan

 o 

-------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

Long Short Term Memory Model, with the same dataset.

In [ ]:
# Mapping chars to ints :
raw_text = raw_text[:500000]
chars = sorted(list(set(raw_text)))
int_chars = {i: c for i, c in enumerate(chars)}
chars_int = {i: c for c, i in enumerate(chars)}

In [ ]:

# Get number of chars and vocab in our text :
n_chars = len(raw_text)
n_vocab = len(chars)

In [ ]:

print('Total Characters : ' , n_chars) 
print('Total Vocabulary : ', n_vocab)

Total Characters :  500000
Total Vocabulary :  34


In [ ]:
# process the dataset, asw
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = raw_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)


Total Patterns :  499900


In [ ]:
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

In [ ]:
LSTM_layer_num = 2 # number of LSTM layers
layer_size = [256,256] # number of nodes in each layer
model = Sequential()

We need to use the 1.15 version of tensorflow --> we might need to restart the kernel.

In [ ]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 512kB 31.5MB/s 
     |████████████████████████████████| 3.8MB 45.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0aa6c7d9554cba6c630e0a71a4fc55d49f15e44fca96f60ef76710cb9cb70de9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: 

In [ ]:
#
import tensorflow.contrib

model.add(CuDNNLSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))
for i in range(1,LSTM_layer_num) :
    model.add(CuDNNLSTM(layer_size[i], return_sequences=True))
model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 256)          265216    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 100, 256)          526336    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 34)                870434    
_________________________________________________________________
activation_1 (Activation)    (None, 34)                0         
Total params: 1,661,986
Trainable params: 1,661,986
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,
          y,
          epochs = 1,
           initial_epoch =0)




Epoch 1/1
499900/499900 [==============================] - 275s 551us/step - loss: 2.6195


In [ ]:
# set a random seed :
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
#print('Seed : ')
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")

# How many characters you want to generate
generated_characters = 400

# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')


" er
 dime que me cres
dime que me cres
dime que sientes cuando me ves
cuando me voy cuando no estoy
d "

e mue te es la darar
co nue de eas a la da lara da ma aada da la aaaaa                                                                                                                                                                                                                                                                                                                                          
Done
